## Aula 14 - Feature Engineering - Parte 2

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from plotting import plot_confusion_matrix

In [ ]:
data = pd.read_csv('data/Base Analytics.csv')

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
data = data[data['Admissão'] >= '2011-01-01'].reset_index().drop(columns='index')
data_num = data.select_dtypes(include=numerics).copy()
data_num = data_num.drop(columns=['ADP', 'Cod.Cargo', 'Cod.Cargo Admissão', 'CC', 
                                              'Hora Extra 2016', 'Hora Negativa 2016', 'Ad. Noturno 2016', 
                                              'Absenteísmo 2016', 'Hora Extra 2017', 'Hora Negativa 2017', 
                                              'Ad. Noturno 2017', 'Absenteísmo 2017', 'Banda', 
                                              '2012/13 Goal Achievement'], axis=1)

In [ ]:
# Preenchendo os nulos
for col in data_num.columns:
    qtt = data_num[col].isnull().sum()
    if qtt > 0:
        print(col, qtt)
        data_num.update(data_num[col].fillna(data_num[col].mean()))

In [ ]:
# Aplicando o StandardScaler 
scaler = StandardScaler()
data_num_scaled = pd.DataFrame(scaler.fit_transform(data_num.drop(columns='PROMOVIDO')),
                               columns=list(data_num.drop(columns='PROMOVIDO').columns))

#### Quais outras features usar?
Quais são os tipos das features desse dataset?

Podemos/queremos usar todas elas? 

.

.

.

.

.

.

.

.

.

In [ ]:
data.columns

In [ ]:
data.isnull().sum()

### Categorical Data

#### One Hot Encoding

![](images/one_hot1.png)
![](images/one_hot2.png)

In [ ]:
le = LabelEncoder()
sex_intlabels = le.fit_transform(data['Sexo'])
data['Sexo_'] = sex_intlabels

In [ ]:
data.sample(5)

In [ ]:
ohe = OneHotEncoder()
sex_cols = ohe.fit_transform(data[['Sexo_']]).toarray()
sex_cols_labels = ['Sexo_'+str(cls_label) for cls_label in set(sex_intlabels)]
sex_df = pd.DataFrame(sex_cols, columns=sex_cols_labels )

In [ ]:
X = pd.concat([data_num_scaled,sex_df], axis=1)
y = data_num['PROMOVIDO']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression(random_state=42, class_weight='balanced')
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
_ = plot_confusion_matrix(y_test, y_pred)
_ = plot_confusion_matrix(y_test, y_pred, normalize='yes')

#### Dummies
Pandas Dataframe function [*get_dummies*](http://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.get_dummies.html)

In [ ]:
# Exercício: 3 min
# Aplique o StandardScaler na coluna 'Sexo' do dataframe data
sex_dummies = 

In [ ]:
X = pd.concat([data_num_scaled,sex_dummies], axis=1)
X.sample(5)

In [ ]:
# Exercício: 3 min
# Crie X e y, separe em train e test e treine a regressão logística
# plote a matriz de confusão do modelo treinado

#### Effect Coding
- Bem similar ao *dummy*;
- Substitui por -1 todos os 0s que representam uma das categorias (primeira ou última) no dummy encoding;
- Exemplo do [link 2](https://towardsdatascience.com/understanding-feature-engineering-part-2-categorical-data-f54324193e63) do pré-aula:

![Example1](images/effect_coding.png)

- [Outro exemplo](https://stats.idre.ucla.edu/other/mult-pkg/faq/general/faqwhat-is-effect-coding/)

#### Bin-counting 
- Maldição da dimensionalidade, categorias "grandes" (com muitos valores distintos);
- Usa probabilidade baseada em informação estatística da relação do valor com o target;
- Exemplo simplificado do [link 2](https://towardsdatascience.com/understanding-feature-engineering-part-2-categorical-data-f54324193e63): construir valores de probabilidade de um ataque DDoS ser causado por qualquer um dos endereços IP baseado em **dados históricos** de endereços IP relacionados a ataques DDoS.

#### Feature Hashing Scheme
- Maldição da dimensionalidade;
- **Função Hash** tipicamente usada com um número pré-definido de features resultantes (vetor de comprimento pré-definido) de forma que os valores de hash dos valores da categoria sejam usados como índices nesse vetor predefinido e os valores sejam atualizados de acordo.
- Funções Hash mapeiam uma grande quantidade de valores em uma quantidade finita (e pequena) de valores -> colisões.
- [Explicação básica sobre funções hash](https://www.techtudo.com.br/artigos/noticia/2012/07/o-que-e-hash.html)
- [Função Scikit Learn](https://scikit-learn.org/0.19/modules/generated/sklearn.feature_extraction.FeatureHasher.html)